# Self Organized Maps (SOMs) - Create Gifs

In [ ]:
observatory = '/home/mfonseca/fastdisk'
my_computer = '/home/polaris/Lab_Astro/data/'

directory = my_computer

## Import Libraries

In [ ]:
import matplotlib.pyplot as plt

# Create GIFs
import imageio.v3
import os
import natsort

import numpy as np
import pandas as pd

from astropy.table import Table

# SimpSOM, https://simpsom.readthedocs.io
import simpsom as sps

# CuPY
# import cupy as cp

## GIF Creator

In [ ]:
def create_gif_from_folder(folder_path, gif_filename="images.gif", duration=0.5):
    """
    Generates a GIF from the ordered images in a folder and saves it in the same folder.

    Args:
        folder_path (str): The path to the folder containing the images.
        gif_filename (str, optional): The name of the output GIF file.
            Defaults to "images.gif".  The file will be saved in the
            same folder as the images.
        duration (float, optional): The duration of each frame in the GIF,
            in seconds. Defaults to 0.5.
    """
    # Get a list of all files in the folder
    files = os.listdir(folder_path)

    # Use natural sort to order the files correctly
    files = natsort.natsorted(files)

    # Filter out non-image files (optional, but recommended)
    image_files = [f for f in files if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]

    if not image_files:
        print(f"Error: No image files found in the folder: {folder_path}")
        return

    # Construct the full path to each image
    image_paths = [os.path.join(folder_path, file) for file in image_files]

    # Read the images using imageio
    images = []
    for image_path in image_paths:
        try:
            images.append(imageio.imread(image_path))
        except ValueError:
            print(f"Warning: Could not read image file: {image_path}. Skipping.")
            continue  # Skip to the next file

    if not images:
        print(f"Error: No readable image files found in the folder: {folder_path}")
        return

    # Construct the full path for the output GIF
    gif_filepath = os.path.join(folder_path, gif_filename)

    # Create the GIF
    imageio.v3.mimsave(gif_filepath, images, duration=duration, loop=0)

    print(f"GIF created successfully: {gif_filepath}")

## Original data

In [ ]:
original_data_path = directory + 'soms/maps1/EMU_0102-32_DESY6GOLD_VIKINGDR5_CATWISE_noMag_zcaWhite.csv'
data_white = pd.read_csv(original_data_path)

## Define folder for maps

In [ ]:
folder_som_epochs = directory + '/home/polaris/Lab_Astro/data/soms/maps1'
feature_columns = ['DESY6_mag_auto_g_extcorr',
                   'VKG_zAperMag3_ab_extcorr',
                   'CAT_w1mpro_ab',
                   'DESY6_g_r_extcorr', 
                   'DESY6_r_i_extcorr', 
                   'DESY6_i_z_extcorr', 
                   'DESY6_z_y_extcorr', 
                   'VKG_z_y_am3_extcorr', 
                   'VKG_y_j_am3_extcorr', 
                   'VKG_j_h_am3_extcorr', 
                   'VKG_h_ks_am3_extcorr', 
                   'CAT_w1_w2_ab',
                   'DESY6_dnf_z', 
                   'DESY6_spread_model_g', 
                   'DESY6_spread_model_r', 
                   'DESY6_spread_model_i', 
                   'DESY6_spread_model_z']

## GIF of SOM by epoch

In [ ]:
num_epochs = 50

for i in range(1,num_epochs):
    name_file = f'{folder_som_epochs}/epoch_files/trained_som_' + str(i) + 'epoch'+ '.npy'
    
    # Define the SOM
    sommap = sps.SOMNet(
    17,
    16,
    data_white,
    load_file = name_file,
    debug= False,
    )

    # Plot and save the som
    som_file = f'{folder_som_epochs}/epochs/trained_som_' + str(i) + 'epoch' + '.png'
    sommap.plot_map_by_difference(show=False, print_out=True, title='SOM PCA Whitened data 50 Epochs', figsize=(8,8), file_name = som_file)

    # Plot and save the features
    for idx in range(len(feature_columns)):
        feature_file = f'{folder_som_epochs}/feature/feature{idx}/feature{i}epoch.png'
        sommap.plot_map_by_feature(
            feature_ix=idx,
            show=False,
            print_out=True,
            cbar_label=feature_columns[i],
            figsize=(10, 10),
            file_name=feature_file,
            title = feature_columns[idx]
        )

## GIF of Features

In [ ]:
create_gif_from_folder(directory+'/soms/maps1/epochs', 'epochs.gif')

## GIF of Quantization Error (QE) and Topographic Error (TE)

In [ ]:
num_epochs = 50

for i in range(1,num_epochs):
    name_file = f'{folder_som_epochs}epoch_files/trained_som_' + str(i) + 'epoch'+ '.npy'
    
    # Define the SOM
    sommap = sps.SOMNet(
    17,
    16,
    data_white,
    load_file = name_file,
    debug= False,
    )

    hexagon = sps.polygons.Hexagons
    all_positions = [[node.pos[0], node.pos[1]] for node in sommap.nodes_list]

    # Plot and save the Quantization Error (QE)
    # For each point calculate the distance to the BMU and save it
    data_white = np.array(data_white)
    bmu_indices = sommap.find_bmu_ix(data_white)
    bmu_weights = np.array([sommap.nodes_list[int(bmu_idx)].weights for bmu_idx in bmu_indices])
    distances_batch = sps.distance.pairdist(data_white, bmu_weights, metric='euclidean')
    individual_qe = np.diag(distances_batch)


    qe_file = f'{folder_som_epochs}qe/qe{i}epoch.png'
    sps.plots.plot_map(
            all_positions,
            individual_qe,
            hexagon,
            show=False,
            print_out=True,
            figsize=(10, 10),
            file_name=qe_file,
            title = 'Quantization Error Map'
        )

    # Plot and save the Topographic Error (TE)
    individual_te = []

    bmu_indices = sommap.find_2bmu_ix(data_white)
    bmu_positions = np.array([sommap.nodes_list[int(bmu_idx)].pos for bmu_idx in bmu_indices[:,0]])
    sbmu_positions = np.array([sommap.nodes_list[int(sbmu_idx)].pos for sbmu_idx in bmu_indices[:,1]])

    nodes_list = sommap.nodes_list

    for node in nodes_list:
        
        bmu_positions_pernode = [bmu_positions[i] for i in range(nodes_list) if bmu_indices[i]==node]
        sbmu_positions_pernode  = [sbmu_positions for i in range(nodes_list) if bmu_indices[i]==node]

        total_friends = len(bmu_positions_pernode)

        bmu_row, bmu_col = bmu_positions_pernode[:, 0], bmu_positions_pernode[:, 1]
        sbmu_row, sbmu_col = sbmu_positions_pernode[:, 0], sbmu_positions_pernode[:, 1]

        # Check if BMU and sBMU are neighbors
        row_neighbors = np.abs(bmu_row - sbmu_row)
        col_neighbors = np.abs(bmu_col - sbmu_col)

        list_not_neighbors = (row_neighbors > 1) | (col_neighbors > 1)

        not_neighbors = np.sum(not_neighbors)

        individual_te.append(not_neighbors/total_friends)

    te_file = f'{folder_som_epochs}te/te{i}epoch.png'
    sps.plots.plot_map(
            all_positions,
            individual_te,
            hexagon,
            show=False,
            print_out=True,
            figsize=(10, 10),
            file_name=te_file,
            title = 'Topographic Error Map'
        )
